In [123]:
import MaterialX as mx
from IPython.display import display, HTML

display(HTML(f"<p><strong>MaterialX Version:</strong> {mx.getVersionString()}</p>"))

import sys
#display(HTML(f"<p><strong>Python Executable:</strong> {sys.executable}</p>"))
#display(HTML(f"<p><strong>Python Path:</strong> {sys.path}</p>"))
#import MaterialX.PyMaterialXRender as mxr

#dir(mxr)

In [124]:
# Check the contents of the MaterialX package and its submodules
import importlib
import pkgutil
from IPython.display import display, HTML

html_output = "<h3>MaterialX package contents:</h3>"
#html_output +="<ul>"
contents = dir(importlib.import_module('MaterialX'))
for item in contents:
    html_output += f"<li>MaterialX.{item}.</li>"
html_output += "</ul>"


#html_output += "<h3>MaterialX package modules:</h3><ul>"
module_list = []
module_list_html = ""
for module_info in pkgutil.iter_modules(importlib.import_module('MaterialX').__path__):
    #html_output += f"<li>{module_info.name}</li>"
    module_list.append(module_info.name)
for module_name in module_list:
    module_list_html += f"<li>{module_name}</li>"    
#html_output += "</ul>"

import os
import MaterialX

module_paths = []
module_path = os.path.join(MaterialX.__path__[0])
for file in os.listdir(module_path):
    module_paths.append(file)
module_path_html = ""
for file in os.listdir(module_path):
    module_path_html += f"<li>{file}</li>"

#html_output += f"<h3>MaterialX Python directory:</h3><p>{module_path}</p>"
html_output += f"<h3>Contents of MaterialX Package:</h3>"
html_output += "<table style='width:100%; border: 1px solid black; border-collapse: collapse;'>"
html_output += "<thead>"
html_output += "<tr><th>Module Names</th><th>Module Folders</th</tr>"
html_output + "</thead><tbody>"
html_output += f"<tr><td><ul>{module_list_html}</ul></td><td><ul>{module_path_html}</ul></td></tr>"
html_output += "</tbody></table>"

display(HTML(html_output))

In [125]:
# Test import of all PyMaterialX extension modules
from IPython.display import display, HTML

html_output = "<h3>Module Import Results:</h3><ul>"
imported_modules = {}  # Store successfully imported modules

for mod in module_list:
    try:
        imported = __import__('MaterialX.' + mod, fromlist=[''])
        imported_modules[mod] = imported  # Save for later use
        html_output += f"<li><b>{mod}</b> : {imported.__doc__ if imported.__doc__ else 'Undocumented'}</li>"
    except Exception as e:
        html_output += f"<li>Failed to import {mod}: {e}</li>"
html_output += "</ul>"

display(HTML(html_output))

In [126]:
# Get detailed help for each module including only items defined in the module (no duplicates)
from IPython.display import display, HTML
import inspect

html_output = "<h3>Detailed Module Information:</h3>"

for mod, imported in imported_modules.items():
    html_output += f"<div style='border: 1px solid #ccc; margin: 10px 0; padding: 10px;'>"
    html_output += f"<h4>MaterialX.{mod}</h4>"
    if imported.__doc__:
        html_output += f"<p><strong>Module Description:</strong> {imported.__doc__}</p>"

    seen = set()
    items = dir(imported)
    classes = []
    functions = []
    for name in items:
        if name in seen or name.startswith('_'):
            continue
        obj = getattr(imported, name, None)
        # Only include if defined in this module
        if inspect.isclass(obj) and getattr(obj, '__module__', None) == imported.__name__:
            #print(f'Found class: {name} in module: {imported.__name__}')
            if name == 'Document':
                print(f'Document class methods: {[n for n, o in inspect.getmembers(obj, inspect.isfunction)]}')
            classes.append((name, obj))
            seen.add(name)
        elif (inspect.isfunction(obj) or inspect.isbuiltin(obj)) and getattr(obj, '__module__', None) == imported.__name__:
            functions.append((name, obj))
            seen.add(name)

    if classes:
        html_output += f"<p><strong>Classes ({len(classes)}):</strong></p>"
        for cls_name, cls_obj in classes:
            doc = cls_obj.__doc__.split('\n')[0] if cls_obj.__doc__ else ''
            methods = [name for name, obj in inspect.getmembers(cls_obj, inspect.isfunction) ] #if not name.startswith('_')]
            if methods:
                html_output += f"<details style='margin-left: 20px;'>"
                html_output += f"<summary><code><strong>{cls_name}</strong></code>: {doc}</summary>"
                html_output += f"<div style='margin-left: 20px; margin-top: 10px;'>"
                html_output += f"<strong>Methods ({len(methods)}):</strong><ul>"
                for method_name in methods:
                    try:
                        #if cls_name == 'Document':
                        #    print(f'Document method: {method_name}')
                        method_obj = getattr(cls_obj, method_name)
                        sig = inspect.signature(method_obj)
                        method_doc = method_obj.__doc__.split('\n')[0] if method_obj.__doc__ else 'No description'
                        html_output += f"<li><code>{method_name}{sig}</code><br/><em>{method_doc}</em></li>"
                    except Exception:
                        html_output += f"<li><code>{method_name}(...)</code></li>"
                html_output += "</ul></div>"
                html_output += "</details>"
            else:
                html_output += f"<div style='margin-left: 30px;'><code><strong>{cls_name}</strong></code>: {doc} (No public methods)</div>"

    if functions:
        html_output += f"<p><strong>Functions ({len(functions)}):</strong></p><ul>"
        for func_name, func_obj in functions:
            try:
                sig = inspect.signature(func_obj)
                doc = func_obj.__doc__.split('\n')[0] if func_obj.__doc__ else 'No description'
                html_output += f"<li><code><strong>{func_name}{sig}</strong></code><br/><em>{doc}</em></li>"
            except Exception:
                html_output += f"<li><code>{func_name}(...)</code>: Unable to get signature</li>"
        html_output += "</ul>"

    # Fallback: If no classes or functions found, show type and help for all public items
    if not classes and not functions:
        html_output += f"<p><strong>No classes or functions found. Showing type and help for public items:</strong></p><ul>"
        for name in items:
            if name.startswith('_'):
                continue
            obj = getattr(imported, name, None)
            try:
                obj_type = type(obj)
                doc = obj.__doc__.split('\n')[0] if obj and hasattr(obj, '__doc__') and obj.__doc__ else ''
                html_output += f"<li><code>{name}</code>: <em>{obj_type}</em><br/><span>{doc}</span></li>"
            except Exception:
                html_output += f"<li><code>{name}</code>: Unable to get type or doc</li>"
        html_output += "</ul>"

    html_output += "</div>"

display(HTML(html_output))

Document class methods: ['addBindInput', 'addBindParam', 'addChild', 'addMaterial', 'addParameter', 'getActiveParameters', 'getBindInputs', 'getBindParams', 'getBindTokens', 'getChild', 'getChildOfType', 'getChildrenOfType', 'getInputValue', 'getMaterials', 'getParameterValue', 'getParameterValueString', 'getParameters', 'isA', 'removeChildOfType', 'setInputValue', 'setParameterValue']


In [127]:
#info = mx.flattenFilenames?

#print(type(mx.flattenFilenames))
#print("call: mx.flattenFilenames?")
#help(mx.flattenFilenames())

import MaterialX as mx

doc = mx.createDocument()
mx.Document?

help(mx)

Help on package MaterialX:

NAME
    MaterialX

DESCRIPTION
    # Python 3.8+ on Windows: DLL search paths for dependent
    # shared libraries
    # Refs.:
    # - https://github.com/python/cpython/issues/80266
    # - https://docs.python.org/3.8/library/os.html#os.add_dll_directory

PACKAGE CONTENTS
    PyMaterialXCore
    PyMaterialXFormat
    PyMaterialXGenGlsl
    PyMaterialXGenMdl
    PyMaterialXGenMsl
    PyMaterialXGenOsl
    PyMaterialXGenShader
    PyMaterialXRender
    PyMaterialXRenderGlsl
    PyMaterialXRenderOsl
    _scripts (package)
    colorspace
    datatype
    main

FUNCTIONS
    createDocument(...) method of builtins.PyCapsule instance
        createDocument() -> MaterialX_v1_39_5::Document
    
    createNamePath(...) method of builtins.PyCapsule instance
        createNamePath(arg0: list[str]) -> str
    
    createValidName(...) method of builtins.PyCapsule instance
        createValidName(name: str, replaceChar: str = '_') -> str
    
    flattenFilenames(...) 

Init signature: mx.Document(self, /, *args, **kwargs)
Docstring:      <no docstring>
File:           c:\users\home\appdata\local\programs\python\python311\lib\site-packages\materialx\pymaterialxcore.cp311-win_amd64.pyd
Type:           pybind11_type
Subclasses:     

In [128]:
# Simple extraction of functions, classes, and class methods per module using dir()
for mod_name, mod in imported_modules.items():
    print(f"\nModule: MaterialX.{mod_name}")
    items = dir(mod)
    functions = []
    classes = []
    for name in items:
        if name.startswith('_'):
            continue
        obj = getattr(mod, name, None)
        if callable(obj):
            if isinstance(obj, type):
                classes.append(name)
            else:
                functions.append(name)
    if classes:
        print(f"  Classes: {classes}")
        for cls_name in classes:
            cls_obj = getattr(mod, cls_name)
            method_names = [m for m in dir(cls_obj) if not m.startswith('_') and callable(getattr(cls_obj, m, None))]
            if method_names:
                print(f"    Methods of {cls_name}: {method_names}")
    if functions:
        print(f"  Functions: {functions}")


Module: MaterialX.PyMaterialXCore
  Classes: ['AttributeDef', 'Backdrop', 'Collection', 'Color3', 'Color4', 'CommentElement', 'Document', 'Edge', 'Element', 'ElementEquivalenceOptions', 'ElementPredicate', 'Exception', 'ExceptionFoundCycle', 'ExceptionOrphanedElement', 'GenericElement', 'GeomElement', 'GeomInfo', 'GeomProp', 'GeomPropDef', 'GraphElement', 'GraphIterator', 'Implementation', 'InheritanceIterator', 'Input', 'InterfaceElement', 'LinearUnitConverter', 'Look', 'LookGroup', 'MaterialAssign', 'Matrix33', 'Matrix44', 'MatrixBase', 'Member', 'NewlineElement', 'Node', 'NodeDef', 'NodeGraph', 'NodePredicate', 'Output', 'PortElement', 'Property', 'PropertyAssign', 'PropertySet', 'PropertySetAssign', 'StringResolver', 'TargetDef', 'Token', 'TreeIterator', 'TypeDef', 'TypedElement', 'TypedValue_boolean', 'TypedValue_booleanarray', 'TypedValue_color3', 'TypedValue_color4', 'TypedValue_float', 'TypedValue_floatarray', 'TypedValue_integer', 'TypedValue_integerarray', 'TypedValue_matrix

In [129]:
# Print functions, classes, and class methods grouped by module name
for mod_name, mod in imported_modules.items():
    print(f"\n{'='*40}\nModule: MaterialX.{mod_name}\n{'='*40}")
    items = dir(mod)
    functions = []
    classes = []
    class_methods = {}
    for name in items:
        if name.startswith('_'):
            continue
        obj = getattr(mod, name, None)
        if callable(obj):
            if isinstance(obj, type):
                classes.append(name)
                # Get methods for this class
                cls_obj = obj
                method_names = [m for m in dir(cls_obj) if not m.startswith('_') and callable(getattr(cls_obj, m, None))]
                if method_names:
                    class_methods[name] = method_names
            else:
                functions.append(name)
    if classes:
        print(f"  Classes: {classes}")
        for cls_name in classes:
            if cls_name in class_methods:
                print(f"    Methods of {cls_name}: {class_methods[cls_name]}")
    if functions:
        print(f"  Functions: {functions}")


Module: MaterialX.PyMaterialXCore
  Classes: ['AttributeDef', 'Backdrop', 'Collection', 'Color3', 'Color4', 'CommentElement', 'Document', 'Edge', 'Element', 'ElementEquivalenceOptions', 'ElementPredicate', 'Exception', 'ExceptionFoundCycle', 'ExceptionOrphanedElement', 'GenericElement', 'GeomElement', 'GeomInfo', 'GeomProp', 'GeomPropDef', 'GraphElement', 'GraphIterator', 'Implementation', 'InheritanceIterator', 'Input', 'InterfaceElement', 'LinearUnitConverter', 'Look', 'LookGroup', 'MaterialAssign', 'Matrix33', 'Matrix44', 'MatrixBase', 'Member', 'NewlineElement', 'Node', 'NodeDef', 'NodeGraph', 'NodePredicate', 'Output', 'PortElement', 'Property', 'PropertyAssign', 'PropertySet', 'PropertySetAssign', 'StringResolver', 'TargetDef', 'Token', 'TreeIterator', 'TypeDef', 'TypedElement', 'TypedValue_boolean', 'TypedValue_booleanarray', 'TypedValue_color3', 'TypedValue_color4', 'TypedValue_float', 'TypedValue_floatarray', 'TypedValue_integer', 'TypedValue_integerarray', 'TypedValue_matrix

In [130]:
# Print functions, classes, and class methods grouped by module name, with docstrings as lists
for mod_name, mod in imported_modules.items():
    print(f"\n{'='*40}\nModule: MaterialX.{mod_name}\n{'='*40}")
    items = dir(mod)
    functions = []
    classes = []
    class_methods = {}
    class_docs = {}
    function_docs = {}
    for name in items:
        if name.startswith('_'):
            continue
        obj = getattr(mod, name, None)
        if callable(obj):
            if isinstance(obj, type):
                classes.append(name)
                class_docs[name] = obj.__doc__.split('\n')[0] if hasattr(obj, '__doc__') and obj.__doc__ else ''
                # Get methods for this class
                cls_obj = obj
                method_names = [m for m in dir(cls_obj) if not m.startswith('_') and callable(getattr(cls_obj, m, None))]
                method_docs = {}
                for m in method_names:
                    method_obj = getattr(cls_obj, m, None)
                    method_docs[m] = method_obj.__doc__.split('\n')[0] if hasattr(method_obj, '__doc__') and method_obj.__doc__ else ''
                if method_names:
                    class_methods[name] = (method_names, method_docs)
            else:
                functions.append(name)
                function_docs[name] = obj.__doc__.split('\n')[0] if hasattr(obj, '__doc__') and obj.__doc__ else ''
    if classes:
        print("  Classes:")
        for cls_name in classes:
            print(f"    - {cls_name}: {class_docs.get(cls_name, '')}")
            if cls_name in class_methods:
                method_names, method_docs = class_methods[cls_name]
                print(f"      Methods of {cls_name}:")
                for m in method_names:
                    print(f"        - {m}: {method_docs.get(m, '')}")
    if functions:
        print("  Functions:")
        for func_name in functions:
            print(f"    - {func_name}: {function_docs.get(func_name, '')}")


Module: MaterialX.PyMaterialXCore
  Classes:
    - AttributeDef: 
      Methods of AttributeDef:
        - addChild: Add a child element of the given subclass, name, and optional type string.
        - addChildOfCategory: addChildOfCategory(self: MaterialX.PyMaterialXCore.Element, category: str, name: str = '') -> MaterialX.PyMaterialXCore.Element
        - asString: asString(self: MaterialX.PyMaterialXCore.Element) -> str
        - changeChildCategory: changeChildCategory(self: MaterialX.PyMaterialXCore.Element, arg0: MaterialX.PyMaterialXCore.Element, arg1: str) -> MaterialX.PyMaterialXCore.Element
        - clearContent: clearContent(self: MaterialX.PyMaterialXCore.Element) -> None
        - copyContentFrom: copyContentFrom(self: MaterialX.PyMaterialXCore.Element, arg0: MaterialX.PyMaterialXCore.Element) -> None
        - createStringResolver: createStringResolver(self: MaterialX.PyMaterialXCore.Element, geom: str = '') -> MaterialX_v1_39_5::StringResolver
        - createValidChil

In [131]:
# Print functions, classes, and class methods grouped by module name, formatted as Markdown
from IPython.display import Markdown, display

total_md = []

for mod_name, mod in imported_modules.items():
    if mod_name.startswith('_'):
        continue
    md = []
    md.append(f"<h3>Module: MaterialX.{mod_name}</h3>")
    items = dir(mod)
    functions = []
    classes = []
    class_methods = {}
    class_docs = {}
    function_docs = {}
    for name in items:
        if name.startswith('_'):
            continue
        obj = getattr(mod, name, None)
        if callable(obj):
            if isinstance(obj, type):
                classes.append(name)
                class_docs[name] = obj.__doc__.split('\n')[0] if hasattr(obj, '__doc__') and obj.__doc__ else ''
                # Get methods for this class
                cls_obj = obj
                method_names = [m for m in dir(cls_obj) if not m.startswith('_') and callable(getattr(cls_obj, m, None))]
                method_docs = {}
                for m in method_names:
                    method_obj = getattr(cls_obj, m, None)
                    method_docs[m] = method_obj.__doc__.split('\n')[0] if hasattr(method_obj, '__doc__') and method_obj.__doc__ else ''
                if method_names:
                    class_methods[name] = (method_names, method_docs)
            else:
                functions.append(name)
                function_docs[name] = obj.__doc__.split('\n')[0] if hasattr(obj, '__doc__') and obj.__doc__ else ''
    if classes:
        md.append("<details><summary>Classes</summary>\n")
        for cls_name in classes:
            doc = class_docs.get(cls_name, '')
            md.append(f"- **{cls_name}**: {doc}")
            if cls_name in class_methods:
                method_names, method_docs = class_methods[cls_name]
                md.append(f"    <details><summary>Methods</summary><ul>")
                #md.append(f"    - Methods of **{cls_name}**:")
                for m in method_names:
                    md.append(f"        <li> `{m}`: {method_docs.get(m, '')}")
                md.append("    </ul></details>")
        md.append("</details>")
    if functions:
        md.append("<details><summary>Functions</summary>")
        md.append("<ul>")
        for func_name in functions:
            doc = function_docs.get(func_name, '')
            md.append(f"<li> `{func_name}`: {doc}")
        md.append("</ul>")
        md.append("</details>")

    total_md.extend(md)

    #display(Markdown('\n'.join(md)))

filename = f"Python_Module_documentation.md"
print("Write docs to file:", filename)
with open(filename, "w", encoding="utf-8") as f:
    f.write('\n'.join(total_md))


Write docs to file: Python_Module_documentation.md
